In [ ]:
import os
import glob
import pandas as pd
import re

In [2]:
conjunto_dados = pd.DataFrame()

SPEAKER_REGEX = re.compile(r'[A|O] Sr\.ª?( [\w\-,]+)+([\-\w]+)?( \([\w\s\-]+\))*(?=:)')
MULTILINE_SPEAKER_REGEX = re.compile("^" + SPEAKER_REGEX.pattern, flags=re.M) 

for file in os.scandir("./discursos"):
    speech_by_speaker = []

    with open(file) as fp:
        content = fp.read()

        #1) remover datas espalhadas, páginas e afins espalhadas pelos documentos (por causa da coversao PDF para TXT)

        datas_texto = re.compile("\d{1,2} [A-Z]+ [A-Z]+ [A-Z]+ \d{4} (\n\n\d+)?")
        datas_texto.sub(" ", content)

        series_numeros = re.compile("[A-Z]+ SÉRIE \W NÚMERO \d+ (\n+\d+)?")
        series_numeros.sub(" ", content)

        #2) criar dicionario de intervencoes para criar dataframe

        #2.1) identificar os intervenientes e intervencoes correspondentes

        deputadxs = MULTILINE_SPEAKER_REGEX.finditer(content) 
        deputadxs = [match.group() for match in deputadxs]
        deputadxs = list(set(deputadxs))

        #deputadxs = [deputadxs[5]]
        #print(deputadxs)

        content_on_one_line = ''.join(content.splitlines())

        for speaker in deputadxs:
            content_split_by_speaker = content_on_one_line.split(speaker + ':')
            # Get rid of everything before they speak
            content_split_by_speaker.pop(0)

            # what_they_say = [speech for speech in speaker_regex]
            # for each line in content_split_by_speaker
            # use the speaker regex to find the next speaker and get everything before that
            for line in content_split_by_speaker:
                start_of_next_speaker = SPEAKER_REGEX.finditer(line)

                # either an index or None
                # if it's None, either:
                # - the regex isn't accurate enough; or
                # - the speaker is the only person to speak]
                next_speaker_indexes = [match.start() for match in start_of_next_speaker]
                next_speaker_index = next_speaker_indexes[0] if len(next_speaker_indexes) > 0 else None

                speech = line[0:next_speaker_index] if next_speaker_index is not None else line
                speech_by_speaker.append((speaker, speech))

                # if "):" in speech:
                #     print(speech)
        
        df = pd.DataFrame.from_records(speech_by_speaker, columns=["speaker", "speech"])#.reset_index()
        df['filename'] = os.path.basename(file)
        #print(df)
                
        conjunto_dados = pd.concat([conjunto_dados, df])
            
conjunto_dados#.to_csv("deputados.csv")#, index=False)
    

,speaker,speech,filename
0,A Sr.ª Ana Mesquita (PCP),"— Sr. Presidente, Sr.as Deputadas e Srs. Depu...",darl13sl03n050.txt
1,A Sr.ª Ana Mesquita (PCP),— A esmagadora maioria da população aprendeu ...,darl13sl03n050.txt
2,A Sr.ª Ana Mesquita (PCP),"— Já houve um, na Legislatura anterior!",darl13sl03n050.txt
3,A Sr.ª Ana Mesquita (PCP),— Esse argumento é falso!,darl13sl03n050.txt
4,O Sr. João Oliveira (PCP),— Exatamente!,darl13sl03n050.txt
...,...,...,...
229,A Sr.ª Carla Cruz (PCP),"— Sr. Presidente, Srs. Deputados: Só há melho...",darl13sl03n058.txt
230,A Sr.ª Carla Cruz (PCP),— É verdade!,darl13sl03n058.txt
231,A Sr.ª Carla Cruz (PCP),"— Sr. Presidente, Srs. Deputados: O CDS e o P...",darl13sl03n058.txt
232,A Sr.ª Carla Cruz (PCP),— Foi essa a política que fizeram ao longo do...,darl13sl03n058.txt


In [3]:
conjunto_dados["speech"] = [string.lstrip("— ") for string in conjunto_dados["speech"]]


partidos_raw = conjunto_dados["speaker"].str.split("(", n=1, expand=True)
partidos_raw.columns = ["speaker", "partido"]
partidos_raw["partido"] = partidos_raw["partido"].str.rstrip(")")
partidos_raw["speech"] = conjunto_dados["speech"]
partidos_raw["filename"] = conjunto_dados["filename"]
partidos_raw["speaker"] = partidos_raw["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")
partidos_raw["speaker"] = partidos_raw["speaker"].str.rstrip(" ")
#conjunto_dados = conjunto_dados.merge(partidos_raw, how="right", on="speaker") it was breaking (maybe memory issues?)

#partidos_raw.to_csv("deputados.csv", index=False)

/tmp/ipykernel_16772/3255187515.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  partidos_raw["speaker"] = partidos_raw["speaker"].str.replace(r"[A|O] Sr\.ª? ", "")


In [4]:
partidos_raw

,speaker,partido,speech,filename
0,Ana Mesquita,PCP,"Sr. Presidente, Sr.as Deputadas e Srs. Deputad...",darl13sl03n050.txt
1,Ana Mesquita,PCP,A esmagadora maioria da população aprendeu a n...,darl13sl03n050.txt
2,Ana Mesquita,PCP,"Já houve um, na Legislatura anterior!",darl13sl03n050.txt
3,Ana Mesquita,PCP,Esse argumento é falso!,darl13sl03n050.txt
4,João Oliveira,PCP,Exatamente!,darl13sl03n050.txt
...,...,...,...,...
229,Carla Cruz,PCP,"Sr. Presidente, Srs. Deputados: Só há melhor s...",darl13sl03n058.txt
230,Carla Cruz,PCP,É verdade!,darl13sl03n058.txt
231,Carla Cruz,PCP,"Sr. Presidente, Srs. Deputados: O CDS e o PSD ...",darl13sl03n058.txt
232,Carla Cruz,PCP,Foi essa a política que fizeram ao longo do vo...,darl13sl03n058.txt


In [5]:
#correções manuais partidos

partidos_raw.loc[partidos_raw["partido"] == "PEN", "partido"] = 'PAN'

partidos_raw.loc[partidos_raw["partido"] == "OS Verdes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "Os verdes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "Os Vedes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "Os Verdes", "partido"] = 'PEV'
partidos_raw.loc[partidos_raw["partido"] == "s Verdes", "partido"] = 'PEV'

partidos_raw.loc[partidos_raw["partido"] == "SD", "partido"] = 'PSD'
partidos_raw.loc[partidos_raw["partido"] == "PD", "partido"] = 'PSD'

partidos_raw.loc[partidos_raw["partido"] == "CDS", "partido"] = 'CDS-PP'
partidos_raw.loc[partidos_raw["partido"] == "CSD-PP", "partido"] = 'CDS-PP'
partidos_raw.loc[partidos_raw["partido"] == "CDS-P", "partido"] = 'CDS-PP'

partidos_raw.loc[partidos_raw["partido"] == "PC", "partido"] = 'PCP'

partidos_raw.loc[partidos_raw["partido"] == "B E", "partido"] = 'BE'
partidos_raw.loc[partidos_raw["partido"] == "Bloco de Esquerda", "partido"] = 'BE'

partidos_raw.loc[partidos_raw["partido"] == "Partido Socialista", "partido"] = 'PS'



In [6]:
print(partidos_raw["partido"].unique())

['PCP' 'PSD' 'CDS-PP' 'BE' 'PEV' 'PS' 'Emília Santos' 'Jorge Lacão'
 'Pedro Alves' None 'CH' 'Maria da Luz Rosinha' 'IL' 'Eduardo Cabrita'
 'PAN' 'Diogo Leão' 'Cláudia Pereira' 'António Filipe' 'André Moz Caldas'
 'Marta Temido' 'Rosa Monteiro' 'Mariana Vieira da Silva'
 'António Carlos Monteiro' 'José de Matos Correia' 'Duarte Pacheco'
 'Mário Belo Morgado' 'Fernando Negrão' 'Jorge Delgado' 'Gabriel Bastos'
 'Sofia Araújo' 'Helga Correia' 'Duarte Cordeiro'
 'António Mendonça Mendes' 'António Costa' 'Ana Mesquita' 'João Galamba'
 'Mário Centeno' 'João Leão' 'José Manuel Pureza'
 'Helena Mesquita Ribeiro' 'Maria de Fátima Fonseca' 'Sandra Pontedeira'
 'Ana Paula Zacarias' 'Nelson de Souza' 'Pedro Nuno Santos'
 'Luís Capoulas Santos' 'Teresa Caeiro' 'Miguel Cabrita'
 'Augusto Santos Silva' 'João Neves' 'Alexandra Leitão' 'Carlos Miguel'
 'Luís Filipe de Castro Mendes' 'Ana Paula Vitorino' 'Moisés Ferreira'
 'Idália Salvador Serrão' 'João Pedro Matos Fernandes'
 'Inês dos Santos Costa' 'M

In [7]:
declaracoes_partido = partidos_raw.loc[(partidos_raw["partido"] == "PS") \
                |(partidos_raw["partido"] == "PSD") \
                |(partidos_raw["partido"] == "CDS-PP") \
                |(partidos_raw["partido"] == "BE") \
                |(partidos_raw["partido"] == "PCP")\
                |(partidos_raw["partido"] == "PEV")\
                |(partidos_raw["partido"] == "PAN")\
                |(partidos_raw["partido"] == "IL") \
                |(partidos_raw["partido"] == "CH") \
                |(partidos_raw["partido"] == "N insc")]

declaracoes_partido.to_csv("so-partidos.csv", index=False)